# Lab 47: Serverless Security Analysis

## Overview

Analyze security threats unique to serverless environments including function vulnerabilities, event injection, permission misconfigurations, and cold start attacks.

**Difficulty**: Intermediate  
**Duration**: 90-120 minutes  
**Prerequisites**: Lab 45 (Cloud Fundamentals), basic serverless knowledge

## Learning Objectives

By the end of this lab, you will be able to:
1. Analyze serverless function logs for security events
2. Detect event injection and data poisoning attacks
3. Investigate IAM permission misconfigurations
4. Identify cold start timing attacks

**Next:** Lab 48 (Cloud IR Automation)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q pandas numpy scikit-learn

In [ ]:
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple
from sklearn.ensemble import IsolationForest

print("Libraries loaded successfully!")

## Part 1: Lambda Log Analysis

Parse and analyze CloudWatch logs from Lambda functions to detect security events.

In [ ]:
def parse_lambda_logs(log_events: List[Dict]) -> pd.DataFrame:
    """Parse CloudWatch logs from Lambda functions."""
    parsed = []
    
    for event in log_events:
        message = event.get('message', '')
        timestamp = datetime.fromtimestamp(
            event.get('timestamp', 0) / 1000
        )
        
        log_entry = {
            'timestamp': timestamp,
            'raw_message': message,
            'log_type': classify_log_type(message)
        }
        
        # Extract request ID
        request_id_match = re.search(
            r'RequestId:\s*([a-f0-9-]+)',
            message
        )
        if request_id_match:
            log_entry['request_id'] = request_id_match.group(1)
        
        # Parse START/END/REPORT lines
        if message.startswith('START'):
            log_entry['event'] = 'invocation_start'
        elif message.startswith('END'):
            log_entry['event'] = 'invocation_end'
        elif message.startswith('REPORT'):
            log_entry['event'] = 'invocation_report'
            log_entry.update(parse_report_line(message))
        else:
            log_entry['event'] = 'application_log'
        
        parsed.append(log_entry)
    
    return pd.DataFrame(parsed)


def parse_report_line(message: str) -> Dict:
    """Extract metrics from REPORT line."""
    metrics = {}
    
    patterns = {
        'duration': r'Duration:\s*([\d.]+)\s*ms',
        'billed_duration': r'Billed Duration:\s*(\d+)\s*ms',
        'memory_size': r'Memory Size:\s*(\d+)\s*MB',
        'memory_used': r'Max Memory Used:\s*(\d+)\s*MB',
        'init_duration': r'Init Duration:\s*([\d.]+)\s*ms'
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, message)
        if match:
            metrics[key] = float(match.group(1))
    
    return metrics


def classify_log_type(message: str) -> str:
    """Classify log message type."""
    if message.startswith(('START', 'END', 'REPORT')):
        return 'platform'
    elif 'ERROR' in message or 'Exception' in message:
        return 'error'
    elif re.match(r'\d{4}-\d{2}-\d{2}', message):
        return 'application'
    else:
        return 'other'

print("Lambda log parser ready!")

In [ ]:
# Demo: Parse Lambda logs
print("LAMBDA LOG PARSING DEMO")
print("=" * 50)

# Simulated CloudWatch log events
sample_log_events = [
    {'timestamp': 1705312800000, 'message': 'START RequestId: abc-123 Version: $LATEST'},
    {'timestamp': 1705312800100, 'message': '2024-01-15T10:00:00.100Z abc-123 INFO Processing request'},
    {'timestamp': 1705312800200, 'message': '2024-01-15T10:00:00.200Z abc-123 INFO Query: SELECT * FROM users'},
    {'timestamp': 1705312800500, 'message': 'END RequestId: abc-123'},
    {'timestamp': 1705312800600, 'message': 'REPORT RequestId: abc-123 Duration: 450.00 ms Billed Duration: 500 ms Memory Size: 128 MB Max Memory Used: 64 MB Init Duration: 150.00 ms'},
    {'timestamp': 1705312801000, 'message': 'START RequestId: def-456 Version: $LATEST'},
    {'timestamp': 1705312801100, 'message': '2024-01-15T10:00:01.100Z def-456 ERROR Connection timeout'},
    {'timestamp': 1705312801200, 'message': 'END RequestId: def-456'},
    {'timestamp': 1705312801300, 'message': 'REPORT RequestId: def-456 Duration: 30000.00 ms Billed Duration: 30000 ms Memory Size: 128 MB Max Memory Used: 100 MB'},
]

# Parse logs
logs_df = parse_lambda_logs(sample_log_events)

print("\nParsed Log Events:")
print(logs_df[['timestamp', 'event', 'log_type']].to_string())

# Extract metrics from REPORT lines
reports = logs_df[logs_df['event'] == 'invocation_report']
print("\n" + "-" * 50)
print("Invocation Metrics:")
for _, report in reports.iterrows():
    print(f"\n  Request: {report.get('request_id', 'N/A')}")
    print(f"    Duration: {report.get('duration', 'N/A')} ms")
    print(f"    Memory Used: {report.get('memory_used', 'N/A')} MB")
    if pd.notna(report.get('init_duration')):
        print(f"    Cold Start: Yes ({report['init_duration']} ms)")
    else:
        print(f"    Cold Start: No")

## Part 2: Invocation Anomaly Detection

Detect anomalous function invocations that may indicate attacks.

In [ ]:
def detect_invocation_anomalies(logs_df: pd.DataFrame) -> pd.DataFrame:
    """Detect anomalous function invocations."""
    # Filter to report lines only
    reports = logs_df[logs_df['event'] == 'invocation_report'].copy()
    
    if len(reports) < 10:
        return pd.DataFrame()
    
    # Aggregate metrics per time window
    reports['hour'] = reports['timestamp'].dt.floor('H')
    
    hourly_stats = reports.groupby('hour').agg({
        'duration': ['mean', 'max', 'std'],
        'memory_used': ['mean', 'max'],
        'request_id': 'count'
    }).reset_index()
    
    hourly_stats.columns = [
        'hour', 'duration_mean', 'duration_max', 'duration_std',
        'memory_mean', 'memory_max', 'invocation_count'
    ]
    hourly_stats = hourly_stats.fillna(0)
    
    # Prepare features for anomaly detection
    features = hourly_stats[[
        'duration_mean', 'duration_max',
        'memory_mean', 'invocation_count'
    ]]
    
    # Detect anomalies with Isolation Forest
    iso = IsolationForest(contamination=0.1, random_state=42)
    hourly_stats['anomaly'] = iso.fit_predict(features)
    
    # Anomalies have score -1
    anomalies = hourly_stats[hourly_stats['anomaly'] == -1]
    
    return anomalies


def detect_error_spikes(logs_df: pd.DataFrame, threshold: float = 3.0) -> pd.DataFrame:
    """Detect sudden increases in error rates."""
    errors = logs_df[logs_df['log_type'] == 'error'].copy()
    
    if len(errors) == 0:
        return pd.DataFrame()
    
    errors['hour'] = errors['timestamp'].dt.floor('H')
    error_counts = errors.groupby('hour').size().reset_index(name='error_count')
    
    # Calculate rolling average
    error_counts['rolling_avg'] = error_counts['error_count'].rolling(
        window=24, min_periods=1
    ).mean()
    
    # Find spikes
    error_counts['is_spike'] = (
        error_counts['error_count'] >
        error_counts['rolling_avg'] * threshold
    )
    
    return error_counts[error_counts['is_spike']]


def analyze_cold_starts(logs_df: pd.DataFrame) -> Dict:
    """Analyze cold start patterns for security implications."""
    reports = logs_df[logs_df['event'] == 'invocation_report'].copy()
    
    if len(reports) == 0:
        return {'status': 'no_data'}
    
    # Identify cold starts (have init_duration)
    reports['is_cold_start'] = reports['init_duration'].notna()
    
    stats = {
        'total_invocations': len(reports),
        'cold_starts': reports['is_cold_start'].sum(),
        'cold_start_rate': reports['is_cold_start'].mean(),
        'avg_cold_start_duration': reports[
            reports['is_cold_start']
        ]['init_duration'].mean() if reports['is_cold_start'].any() else 0,
        'avg_warm_duration': reports[
            ~reports['is_cold_start']
        ]['duration'].mean() if (~reports['is_cold_start']).any() else 0
    }
    
    return stats

print("Invocation anomaly detector ready!")

In [ ]:
# Demo: Cold start analysis
print("COLD START ANALYSIS DEMO")
print("=" * 50)

# Create more extensive log data
base_time = datetime(2024, 1, 15, 10, 0, 0)
extended_logs = []

for i in range(100):
    ts = base_time + timedelta(minutes=i)
    is_cold = i % 20 == 0  # Cold start every 20th invocation
    
    report_msg = f'REPORT RequestId: req-{i:03d} Duration: {np.random.uniform(50, 500):.2f} ms Billed Duration: 500 ms Memory Size: 128 MB Max Memory Used: {np.random.randint(40, 100)} MB'
    if is_cold:
        report_msg += f' Init Duration: {np.random.uniform(100, 300):.2f} ms'
    
    extended_logs.append({
        'timestamp': int(ts.timestamp() * 1000),
        'message': report_msg
    })

# Parse logs
extended_df = parse_lambda_logs(extended_logs)

# Analyze cold starts
cold_start_stats = analyze_cold_starts(extended_df)

print("\nCold Start Statistics:")
print("-" * 50)
print(f"  Total Invocations: {cold_start_stats['total_invocations']}")
print(f"  Cold Starts: {cold_start_stats['cold_starts']}")
print(f"  Cold Start Rate: {cold_start_stats['cold_start_rate']:.1%}")
print(f"  Avg Cold Start Duration: {cold_start_stats['avg_cold_start_duration']:.2f} ms")
print(f"  Avg Warm Duration: {cold_start_stats['avg_warm_duration']:.2f} ms")

## Part 3: Event Injection Detection

Detect injection attacks in event payloads from API Gateway, S3, and other event sources.

In [ ]:
class EventInjectionDetector:
    """Detect injection attacks in serverless event payloads."""
    
    INJECTION_PATTERNS = {
        'sql_injection': [
            r"(?i)(\b(SELECT|INSERT|UPDATE|DELETE|DROP|UNION|ALTER)\b.*\b(FROM|INTO|SET|TABLE)\b)",
            r"(?i)('\s*(OR|AND)\s*'?\d+'\s*=\s*'?\d+)",
            r"(?i)(--\s*$|;\s*--)",
        ],
        'command_injection': [
            r"[;&|`$]",
            r"(?i)\$\(.*\)",
            r"(?i)\b(cat|ls|pwd|whoami|id|wget|curl)\b\s",
        ],
        'path_traversal': [
            r"\.\./",
            r"\.\.\\",
            r"%2e%2e[/\\]",
        ],
        'xss': [
            r"<script[^>]*>",
            r"javascript:",
            r"on\w+\s*=",
        ],
        'ssrf': [
            r"(?i)(localhost|127\.0\.0\.1|0\.0\.0\.0)",
            r"(?i)169\.254\.169\.254",  # AWS metadata
            r"(?i)metadata\.google",     # GCP metadata
        ]
    }
    
    def __init__(self):
        self.findings = []
    
    def analyze_event(self, event: Dict, request_id: str = None) -> List[Dict]:
        """Analyze an event payload for injection patterns."""
        findings = []
        
        # Convert event to string for pattern matching
        payload = json.dumps(event)
        
        for attack_type, patterns in self.INJECTION_PATTERNS.items():
            for pattern in patterns:
                if re.search(pattern, payload):
                    finding = {
                        'timestamp': datetime.now(),
                        'request_id': request_id,
                        'attack_type': attack_type,
                        'pattern': pattern,
                        'payload_preview': payload[:200]
                    }
                    findings.append(finding)
                    self.findings.append(finding)
                    break
        
        return findings
    
    def get_summary(self) -> Dict:
        """Get summary of detected injections."""
        if not self.findings:
            return {'total': 0}
        
        by_type = {}
        for finding in self.findings:
            attack_type = finding['attack_type']
            by_type[attack_type] = by_type.get(attack_type, 0) + 1
        
        return {
            'total': len(self.findings),
            'by_type': by_type
        }

print("Event injection detector ready!")

In [ ]:
# Demo: Event injection detection
print("EVENT INJECTION DETECTION DEMO")
print("=" * 50)

detector = EventInjectionDetector()

# Sample API Gateway events (mix of normal and malicious)
sample_events = [
    # Normal request
    {
        'httpMethod': 'GET',
        'path': '/users/123',
        'queryStringParameters': {'page': '1', 'limit': '10'}
    },
    # SQL injection in query parameter
    {
        'httpMethod': 'GET',
        'path': '/users',
        'queryStringParameters': {'id': "1' OR '1'='1"}
    },
    # Command injection in body
    {
        'httpMethod': 'POST',
        'path': '/process',
        'body': '{"filename": "test.txt; cat /etc/passwd"}'
    },
    # Path traversal
    {
        'httpMethod': 'GET',
        'path': '/files/../../../etc/passwd',
        'queryStringParameters': None
    },
    # SSRF attempt
    {
        'httpMethod': 'POST',
        'path': '/fetch',
        'body': '{"url": "http://169.254.169.254/latest/meta-data/iam/security-credentials/"}'
    },
    # Normal request
    {
        'httpMethod': 'POST',
        'path': '/users',
        'body': '{"name": "John Doe", "email": "john@example.com"}'
    },
]

print("\nAnalyzing events...")
print("-" * 50)

for i, event in enumerate(sample_events):
    findings = detector.analyze_event(event, f'req-{i:03d}')
    
    print(f"\nEvent {i+1}: {event.get('httpMethod')} {event.get('path')}")
    if findings:
        for finding in findings:
            print(f"  ALERT: {finding['attack_type']} detected!")
    else:
        print(f"  Status: Clean")

# Summary
print("\n" + "=" * 50)
print("DETECTION SUMMARY")
summary = detector.get_summary()
print(f"Total Attacks Detected: {summary['total']}")
print(f"By Type: {summary.get('by_type', {})}")

## Part 4: Permission Analysis

Analyze Lambda function IAM roles for over-permissive configurations.

In [ ]:
class FunctionPermissionAnalyzer:
    """Analyze Lambda function IAM role permissions."""
    
    SENSITIVE_PERMISSIONS = {
        'critical': [
            'iam:*',
            '*:*',
            'sts:AssumeRole',
            'organizations:*',
            'kms:Decrypt',
        ],
        'high': [
            's3:*',
            'dynamodb:*',
            'secretsmanager:GetSecretValue',
            'ssm:GetParameter*',
            'lambda:InvokeFunction',
            'ec2:*',
        ],
        'medium': [
            's3:GetObject',
            's3:PutObject',
            'dynamodb:GetItem',
            'dynamodb:PutItem',
            'logs:*',
        ]
    }
    
    def __init__(self):
        self.findings = []
    
    def analyze_function(self, function_config: Dict) -> Dict:
        """Analyze a function's permissions."""
        function_name = function_config.get('function_name', 'unknown')
        policies = function_config.get('policies', [])
        
        result = {
            'function_name': function_name,
            'critical_permissions': [],
            'high_permissions': [],
            'medium_permissions': [],
            'is_overprivileged': False,
            'risk_score': 0
        }
        
        for policy in policies:
            for statement in policy.get('Statement', []):
                actions = statement.get('Action', [])
                if isinstance(actions, str):
                    actions = [actions]
                
                for action in actions:
                    for severity, patterns in self.SENSITIVE_PERMISSIONS.items():
                        if self._matches_pattern(action, patterns):
                            result[f'{severity}_permissions'].append(action)
        
        # Calculate risk score
        result['risk_score'] = (
            len(result['critical_permissions']) * 30 +
            len(result['high_permissions']) * 15 +
            len(result['medium_permissions']) * 5
        )
        
        # Determine if overprivileged
        result['is_overprivileged'] = (
            len(result['critical_permissions']) > 0 or
            len(result['high_permissions']) > 5
        )
        
        self.findings.append(result)
        return result
    
    def _matches_pattern(self, action: str, patterns: List[str]) -> bool:
        """Check if action matches any pattern."""
        for pattern in patterns:
            # Convert wildcard pattern to regex
            regex_pattern = pattern.replace('*', '.*')
            if re.match(regex_pattern, action, re.IGNORECASE):
                return True
        return False
    
    def get_summary(self) -> Dict:
        """Get summary of permission analysis."""
        if not self.findings:
            return {'functions_analyzed': 0}
        
        return {
            'functions_analyzed': len(self.findings),
            'overprivileged': sum(1 for f in self.findings if f['is_overprivileged']),
            'avg_risk_score': np.mean([f['risk_score'] for f in self.findings]),
            'max_risk_score': max(f['risk_score'] for f in self.findings)
        }

print("Permission analyzer ready!")

In [ ]:
# Demo: Permission analysis
print("FUNCTION PERMISSION ANALYSIS DEMO")
print("=" * 50)

analyzer = FunctionPermissionAnalyzer()

# Sample function configurations
sample_functions = [
    {
        'function_name': 'api-handler',
        'policies': [
            {
                'Statement': [
                    {'Action': ['logs:CreateLogGroup', 'logs:PutLogEvents']},
                    {'Action': 'dynamodb:GetItem'}
                ]
            }
        ]
    },
    {
        'function_name': 'data-processor',
        'policies': [
            {
                'Statement': [
                    {'Action': 's3:*'},  # Overly permissive!
                    {'Action': 'dynamodb:*'},
                    {'Action': 'secretsmanager:GetSecretValue'}
                ]
            }
        ]
    },
    {
        'function_name': 'admin-function',
        'policies': [
            {
                'Statement': [
                    {'Action': '*:*'},  # DANGER: Full access!
                    {'Action': 'iam:*'}
                ]
            }
        ]
    }
]

print("\nAnalyzing function permissions...")
print("-" * 50)

for func in sample_functions:
    result = analyzer.analyze_function(func)
    
    print(f"\nFunction: {result['function_name']}")
    print(f"  Risk Score: {result['risk_score']}")
    print(f"  Overprivileged: {result['is_overprivileged']}")
    
    if result['critical_permissions']:
        print(f"  CRITICAL: {result['critical_permissions']}")
    if result['high_permissions']:
        print(f"  HIGH: {result['high_permissions']}")

# Summary
print("\n" + "=" * 50)
print("ANALYSIS SUMMARY")
summary = analyzer.get_summary()
print(f"Functions Analyzed: {summary['functions_analyzed']}")
print(f"Overprivileged Functions: {summary['overprivileged']}")
print(f"Average Risk Score: {summary['avg_risk_score']:.1f}")
print(f"Max Risk Score: {summary['max_risk_score']}")

## Part 5: API Gateway Log Analysis

Analyze API Gateway access logs for security events.

In [ ]:
def analyze_api_gateway_logs(logs_df: pd.DataFrame) -> Dict:
    """Analyze API Gateway logs for security events."""
    security_events = []
    
    for _, log in logs_df.iterrows():
        indicators = []
        
        # Check for error status codes
        status = log.get('status', 0)
        if status >= 400:
            indicators.append(f'error_status_{status}')
        
        # Check for unusual HTTP methods
        method = log.get('http_method', '')
        if method not in ['GET', 'POST', 'PUT', 'DELETE', 'PATCH']:
            indicators.append(f'unusual_method_{method}')
        
        # Check for suspicious paths
        path = log.get('path', '')
        suspicious_paths = ['/admin', '/.env', '/config', '/debug', '/actuator', '/.git']
        if any(sp in path for sp in suspicious_paths):
            indicators.append('suspicious_path')
        
        if indicators:
            security_events.append({
                'timestamp': log.get('timestamp'),
                'source_ip': log.get('source_ip'),
                'path': path,
                'status': status,
                'indicators': indicators
            })
    
    return {'security_events': security_events, 'total': len(security_events)}


def detect_enumeration_attempts(
    logs_df: pd.DataFrame,
    window_minutes: int = 5,
    threshold: int = 20
) -> pd.DataFrame:
    """Detect path/parameter enumeration attempts."""
    logs_df = logs_df.copy()
    logs_df['window'] = logs_df['timestamp'].dt.floor(f'{window_minutes}T')
    
    # Group by source IP and time window
    grouped = logs_df.groupby(['source_ip', 'window']).agg({
        'path': 'nunique',
        'request_id': 'count',
        'status': lambda x: (x == 404).sum()
    }).reset_index()
    
    grouped.columns = ['source_ip', 'window', 'unique_paths',
                       'request_count', 'not_found_count']
    
    # Detect enumeration: many unique paths or many 404s
    enumeration = grouped[
        (grouped['unique_paths'] > threshold) |
        (grouped['not_found_count'] > threshold)
    ]
    
    return enumeration

print("API Gateway log analyzer ready!")

In [ ]:
# Demo: API Gateway log analysis
print("API GATEWAY LOG ANALYSIS DEMO")
print("=" * 50)

# Create sample API Gateway logs
base_time = datetime(2024, 1, 15, 10, 0, 0)

api_logs = pd.DataFrame([
    {'timestamp': base_time, 'source_ip': '10.0.0.1', 'http_method': 'GET', 'path': '/api/users', 'status': 200, 'request_id': 'r1'},
    {'timestamp': base_time + timedelta(seconds=1), 'source_ip': '10.0.0.1', 'http_method': 'POST', 'path': '/api/users', 'status': 201, 'request_id': 'r2'},
    {'timestamp': base_time + timedelta(seconds=2), 'source_ip': '192.168.1.100', 'http_method': 'GET', 'path': '/.env', 'status': 404, 'request_id': 'r3'},
    {'timestamp': base_time + timedelta(seconds=3), 'source_ip': '192.168.1.100', 'http_method': 'GET', 'path': '/admin/config', 'status': 403, 'request_id': 'r4'},
    {'timestamp': base_time + timedelta(seconds=4), 'source_ip': '192.168.1.100', 'http_method': 'OPTIONS', 'path': '/.git/config', 'status': 404, 'request_id': 'r5'},
    {'timestamp': base_time + timedelta(seconds=5), 'source_ip': '10.0.0.2', 'http_method': 'GET', 'path': '/api/products', 'status': 200, 'request_id': 'r6'},
    {'timestamp': base_time + timedelta(seconds=6), 'source_ip': '192.168.1.100', 'http_method': 'GET', 'path': '/debug/vars', 'status': 404, 'request_id': 'r7'},
])

# Analyze logs
analysis = analyze_api_gateway_logs(api_logs)

print("\nSecurity Events Detected:")
print("-" * 50)
print(f"Total: {analysis['total']}")

for event in analysis['security_events']:
    print(f"\n  IP: {event['source_ip']}")
    print(f"  Path: {event['path']}")
    print(f"  Status: {event['status']}")
    print(f"  Indicators: {event['indicators']}")

## Key Takeaways

1. **Log Analysis** - Parse Lambda logs to extract invocation metrics and detect anomalies
2. **Cold Start Monitoring** - Track cold start patterns that may indicate probing
3. **Event Injection Detection** - Scan event payloads for SQL injection, XSS, SSRF
4. **Permission Analysis** - Identify overprivileged function roles
5. **API Gateway Security** - Monitor for enumeration and suspicious access patterns

## Serverless Security Checklist

| Area | Check |
|------|-------|
| Permissions | Least privilege IAM roles |
| Input Validation | Sanitize all event inputs |
| Secrets | Use Secrets Manager, not env vars |
| Dependencies | Scan for vulnerable packages |
| Logging | Enable detailed logging |
| Monitoring | Set up invocation anomaly alerts |

## Next Steps

- **Lab 48**: Cloud IR Automation - Automated incident response
- **Lab 49**: LLM Red Teaming - Advanced security testing
- **Lab 21**: Threat Intelligence - Enrich findings with TI